<a href="https://colab.research.google.com/github/Ethancatepete/Project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install streamlit -q

In [23]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image


st.set_option('deprecation.showfileUploaderEncoding', False)

@st.cache(allow_output_mutation=True)
def load_model():
	model = tf.keras.models.load_model('/content/drive/MyDrive/Models/Nail_Model2.h5')
	return model


def predict_class(image, model):
	image = tf.cast(image, tf.float32)
	image = tf.image.resize(image, [128, 128])
	image = np.expand_dims(image, axis = 0)
	prediction = model.predict(image)

	return prediction

model = load_model()

st.title('Nail Classifier')
st.write(
	"""
	This model is a one hot encoding, which detects whether you have 
	Acral lentiginous melanoma, Beau's lines, Clubbed nails, 
  Cyanosis, Koilonychia - spoon nails, Leukonychia - white nails, 
  Paronychia, Psoriasis, Yellow Nails or Normal Nails. 

	"""
)




file = st.file_uploader("Upload an image of a nail", type=["jpg", "png"])


if file is None:
	st.text('Waiting for upload....')

else:
	slot = st.empty()
	slot.text('Running inference....')

	test_image = Image.open(file)

	st.image(test_image, caption="Input Image", width = 200)

	pred = predict_class(np.asarray(test_image), model)

	nail_names = ['Acral lentiginous melanoma', "Beau's lines", 'Clubbed nails', 
                  'Cyanosis', 'Koilonychia - spoon nails', 'Leukonychia - white nails', 
                  'Normal Nails', 'Paronychia', 'Psoriasis', 'Yellow Nails']


	advice = {
          nail_names[0]: "You should see a doctor urgently, as if this spreads further, you will have skin cancer.",
          nail_names[1]: "You may have chronic diseases, such as uncontrolled diabetes and peripheral vascular disease.",
          nail_names[2]: "You should go to a dermotologists if you are feeling unwell.",
          nail_names[3]: "You do not have enough oxygen in your blood. This means that you have problems with your circulatory system.",
          nail_names[4]: "You may have trauma and may have iron deficiency anemia",
          nail_names[5]: "You lack calcium, zinc so you drink more milk.",
          nail_names[6]: "Your nails are fine.",
          nail_names[7]: "You should see a dermatologist, it means you have an infection, which will become serious.",
          nail_names[8]: "This is a chronic disease. Without treatment, it can be painful, interfere with sleep and make it hard to concentrate.",
          nail_names[9]: "You most likely have a respitory disease."
				}


	index = np.argmax(pred)

	output = (f"The model predicts you have {nail_names[index]}; {advice.get(nail_names[index])}")

	slot.text('Done')

	st.success(output)

Overwriting app.py


In [20]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=db66b50198ea2c81ad599e75d09b2f35e2390da2bf9d29b4b1972a0e44e9a665
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [26]:
from pyngrok import ngrok
ngrok.set_auth_token()#Add your token

In [27]:
!nohup streamlit run app.py --server.port 8080 &
url = ngrok.connect(port='8080')

print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://dd01-34-90-32-106.ngrok.io" -> "http://localhost:80"


In [25]:
ngrok.kill()